Exercise 1: Launch notebook and try exercises embedded in the notebook.

# Erdos-Renyi Graphs

Code examples from [Think Complexity, 2nd edition](https://thinkcomplex.com).

Copyright 2016 Allen Downey, [MIT License](http://opensource.org/licenses/MIT)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns
import random

from utils import decorate, savefig

# I set the random seed so the notebook 
# produces the same results every time.
np.random.seed(17)

# TODO: remove this when NetworkX is fixed
from warnings import simplefilter
import matplotlib.cbook
simplefilter("ignore", matplotlib.cbook.mplDeprecation)

In [ ]:
# node colors for drawing networks
colors = sns.color_palette('pastel', 5)
#sns.palplot(colors)
sns.set_palette(colors)

## Directed graph

The first example is a directed graph that represents a social network with three nodes.

In [ ]:
G = nx.DiGraph()
G.add_node('Alice')
G.add_node('Bob')
G.add_node('Chuck')
list(G.nodes())

Here's how we add edges between nodes.

In [ ]:
G.add_edge('Alice', 'Bob')
G.add_edge('Alice', 'Chuck')
G.add_edge('Bob', 'Alice')
G.add_edge('Bob', 'Chuck')
list(G.edges())

And here's how to draw the graph.

In [ ]:
nx.draw_circular(G,
                 node_color='C0',
                 node_size=2000, 
                 with_labels=True)
plt.axis('equal')

**Exercise:**  Add another node and a few more edges and draw the graph again.

In [ ]:
#Solution to exercise

G.add_node("Debbie") #Add new node Debbie
G.add_edge("Debbie", "Chuck") #Add a directed edge connecting Debbie to Chuck
G.add_edge("Alice","Debbie") #Add a directed edge connecting Alice to Debbie

nx.draw_circular(G,
                 node_color='C0',
                 node_size=2000, 
                 with_labels=True) #Draw the network
plt.axis('equal')

list(G.edges())

## Undirected graph

The second example is an undirected graph that represents cities and the driving times between them.

`positions` is a dictionary that maps from each city to its coordinates.

In [ ]:
positions = dict(Albany=(-74, 43),
                 Boston=(-71, 42),
                 NYC=(-74, 41),
                 Philly=(-75, 40))

positions['Albany']

We can use the keys in `pos` to add nodes to the graph.

In [ ]:
G = nx.Graph()
G.add_nodes_from(positions)
G.nodes()

`drive_times` is a dictionary that maps from pairs of cities to the driving times between them.

In [ ]:
drive_times = {('Albany', 'Boston'): 3,
               ('Albany', 'NYC'): 4,
               ('Boston', 'NYC'): 4,
               ('NYC', 'Philly'): 2}

We can use the keys from `drive_times` to add edges to the graph.

In [ ]:
G.add_edges_from(drive_times)
G.edges()

Now we can draw the graph using `positions` to indicate the positions of the nodes, and `drive_times` to label the edges.

In [ ]:
nx.draw(G, positions, 
        node_color='C1', 
        node_shape='s', 
        node_size=2500, 
        with_labels=True)

nx.draw_networkx_edge_labels(G, positions, 
                             edge_labels=drive_times)

plt.axis('equal')

**Exercise:**  Add another city and at least one edge.

In [ ]:
# Solution goes here

## Complete graph

To make a complete graph, we use a generator function that iterates through all pairs of nodes.

In [ ]:
def all_pairs(nodes):
    for i, u in enumerate(nodes):
        for j, v in enumerate(nodes):
            if i < j:
                yield u, v

`make_complete_graph` makes a `Graph` with the given number of nodes and edges between all pairs of nodes.

In [ ]:
def make_complete_graph(n):
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(all_pairs(nodes))
    return G

Here's a complete graph with 10 nodes:

In [ ]:
complete = make_complete_graph(10)
complete.number_of_nodes()

And here's what it looks like.

In [ ]:
nx.draw_circular(complete, 
                 node_color='C2', 
                 node_size=1000, 
                 with_labels=True)

The `neighbors` method the neighbors for a given node.

In [ ]:
list(complete.neighbors(0))

**Exercise:** Make and draw complete directed graph with 5 nodes.

In [ ]:
# Solution goes here
def double_pairs(nodes):
    """
    The function in the example only creates a single edge if
    a directed graph is used. This function generates all
    possible pairs of nodes (excluding self-pairing).
    """
    nodes = [i for i in range(5)]
    for i, u in enumerate(nodes):
        for j, v in enumerate(nodes):
            if i != j:
                yield(u, v)
                
def make_complete_graph_directed(n):
    """
    This function is a modification of the one in the example.
    This one uses the double_pairs() function we wrote.
    The graph here is also specified to be a directed graph.
    """
    G = nx.DiGraph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(double_pairs(nodes))
    return G

complete = make_complete_graph_directed(5)
complete.number_of_nodes()

nx.draw_circular(complete, 
                 node_color='C2', 
                 node_size=1000, 
                 with_labels=True)


These are all the edges, we see that there's an edge corresponding to both directions

In [ ]:
complete.edges

## Random graphs

Next we'll make a random graph where the probability of an edge between each pair of nodes is $p$.

The helper function `flip` returns True with probability `p` and False with probability `1-p`

In [ ]:
def flip(p):
    return np.random.random() < p

`random_pairs` is a generator function that enumerates all possible pairs of nodes and yields each one with probability `p` 

In [ ]:
def random_pairs(nodes, p):
    for edge in all_pairs(nodes):
        if flip(p):
            yield edge

`make_random_graph` makes an ER graph where the probability of an edge between each pair of nodes is `p`.

In [ ]:
def make_random_graph(n, p):
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(random_pairs(nodes, p))
    return G

Here's an example with `n=10` and `p=0.3`

In [ ]:
np.random.seed(10)

random_graph = make_random_graph(10, 0.3)
len(random_graph.edges())

And here's what it looks like:

In [ ]:
nx.draw_circular(random_graph, 
                 node_color='C3', 
                 node_size=1000, 
                 with_labels=True)

## Connectivity

To check whether a graph is connected, we'll start by finding all nodes that can be reached, starting with a given node:

In [ ]:
def reachable_nodes(G, start):
    seen = set()
    stack = [start]
    while stack:
        node = stack.pop()
        if node not in seen:
            seen.add(node)
            stack.extend(G.neighbors(node))
    return seen

In the complete graph, starting from node 0, we can reach all nodes:

In [ ]:
reachable_nodes(complete, 0)

In the random graph we generated, we can also reach all nodes (but that's not always true):

In [ ]:
reachable_nodes(random_graph, 0)

We can use `reachable_nodes` to check whether a graph is connected:

In [ ]:
def is_connected(G):
    start = next(iter(G))
    reachable = reachable_nodes(G, start)
    return len(reachable) == len(G)

Again, the complete graph is connected:

In [ ]:
is_connected(complete)

But if we generate a random graph with a low value of `p`, it's not:

In [ ]:
random_graph = make_random_graph(10, 0.1)
len(random_graph.edges())

In [ ]:
is_connected(random_graph)

**Exercise:** What do you think it means for a directed graph to be connected?  Write a function that checks whether a directed graph is connected.

According to the book, "A graph is connected if there is a path from every node to every other node". 

For a directed graph, there are three classifications: weakly connected, strongly connected, and unilaterally connected (semiconnected).

A weakly connected graph can be checked with the earlier function "is_connected"

Consider this ring network. Each node has 2 neighbors but you can only go in one direction.

In [ ]:
direct_test = nx.DiGraph()
nodes = range(5)
direct_test.add_nodes_from(nodes)
direct_test.add_edge(0,1)
direct_test.add_edge(1,2)
direct_test.add_edge(2,3)
direct_test.add_edge(3,4)
direct_test.add_edge(4,0)

nx.draw_circular(direct_test,
                 node_color='C0',
                 node_size=2000, 
                 with_labels=True)
plt.axis('equal')

In [ ]:
list(direct_test.neighbors(0))

In [ ]:
list(direct_test.successors(0))

In [ ]:
list(direct_test.to_undirected().neighbors(0))

We see that in networkx, the only "neighbors" considered is the outgoing nodes with a connected. If the test network was undirected, 0 has neighbors 1 and 4. But if we consider direction, 0 can only go to 1. We can reuse the reachable_nodes function to create a new function that checks connectedness for a directed graph.

We'll create a function that classifies if a directed graph is weakly-connected, strongly connected, or not connected.

In [ ]:
reachable_nodes(direct_test.to_undirected(), 0)

In [ ]:
def is_connected_directed(network):
    for node in network:
        reachable = reachable_nodes(network, node)
        if len(reachable) < len(network):
            reachable = reachable_nodes(network.to_undirected(), node)
            if len(reachable) == len(network):
                return "Weakly connected"
            else:
                return False
        elif len(reachable) == len(network):
            return "Strongly connected"

In [ ]:
is_connected_directed(direct_test)

We expect this since we can traverse all nodes from any point in the network if it's arranged in a ring. If we flip one edge, we expect to get a weakly connected directed graph.

If we flip the edge from 1 to 2 to the inverse direction:

In [ ]:
direct_test = nx.DiGraph()
nodes = range(5)
direct_test.add_nodes_from(nodes)
direct_test.add_edge(0,1)
direct_test.add_edge(2,1)
direct_test.add_edge(2,3)
direct_test.add_edge(3,4)
direct_test.add_edge(4,0)

nx.draw_circular(direct_test,
                 node_color='C0',
                 node_size=2000, 
                 with_labels=True)
plt.axis('equal')

In [ ]:
is_connected_directed(direct_test)

Removing two edges should give us a disconnected directed graph

In [ ]:
direct_test = nx.DiGraph()
nodes = range(5)
direct_test.add_nodes_from(nodes)
direct_test.add_edge(0,1)
direct_test.add_edge(2,3)
direct_test.add_edge(3,4)


is_connected_directed(direct_test)

In [ ]:
nx.draw_circular(direct_test,
                 node_color='C0',
                 node_size=2000, 
                 with_labels=True)
plt.axis('equal')

## Probability of connectivity

Now let's estimare the probability that a randomly-generated ER graph is connected.

This function takes `n` and `p`, generates `iters` graphs, and returns the fraction of them that are connected.

In [ ]:
# version with a for loop

def prob_connected(n, p, iters=100):
    count = 0
    for i in range(iters):
        random_graph = make_random_graph(n, p)
        if is_connected(random_graph):
            count += 1
    return count/iters

In [ ]:
# version with a list comprehension

def prob_connected(n, p, iters=100):
    tf = [is_connected(make_random_graph(n, p))
          for i in range(iters)]
    return np.mean(tf)

With `n=10` and `p=0.23`, the probability of being connected is about 33%.

In [ ]:
np.random.seed(17)

n = 10
prob_connected(n, 0.23, iters=10000)

According to Erdos and Renyi, the critical value of `p` for `n=10` is about 0.23. 

In [ ]:
pstar = np.log(n) / n
pstar

So let's plot the probability of connectivity for a range of values for `p`

In [ ]:
ps = np.logspace(-1.3, 0, 11)
ps

I'll estimate the probabilities with `iters=1000`

In [ ]:
ys = [prob_connected(n, p, 1000) for p in ps]

for p, y in zip(ps, ys):
    print(p, y)

And then plot them, adding a vertical line at the computed critical value

In [ ]:
plt.axvline(pstar, color='gray')
plt.plot(ps, ys, color='green')
decorate(xlabel='Prob of edge (p)',
                 ylabel='Prob connected',
                 xscale='log')

We can run the same analysis for a few more values of `n`.

In [ ]:
ns = [300, 100, 30]
ps = np.logspace(-2.5, 0, 11)

sns.set_palette('Blues_r', 4)
for n in ns:
    print(n)
    pstar = np.log(n) / n
    plt.axvline(pstar, color='gray', alpha=0.3)

    ys = [prob_connected(n, p) for p in ps]
    plt.plot(ps, ys, label='n=%d' % n)

decorate(xlabel='Prob of edge (p)',
         ylabel='Prob connected',
         xscale='log', 
         xlim=[ps[0], ps[-1]],
         loc='upper left')

As `n` increases, the critical value gets smaller and the transition gets more abrupt.